Importar y descomprimir

In [2]:
#importar los siguientes módulos.
import os
import json
import pickle
import zipfile
import pandas as pd
import tf_slim

In [3]:
#descomprimir el dataset original.
local_zip = './dataset_original.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('dataset_original')
zip_ref.close()

De JSON a CSV

In [6]:
#ejecutar dos veces, una para test y otra para train.
type_file = 'train'
path = './cv_train.json'
data_file = open(path)
data = json.load(data_file)

In [7]:
#definir la estructura del JSON de origen y de la variables relevantes:
#1. clase.
#2. bounding box (x inicial, y inicial, x final, y final).
#3. bncho y alto de la imagen.
#4. nombre del archivo.
csv_list = []

for classification in data:
  width, height = classification['width'], classification['height']
  image = classification['image']
  for item in classification['tags']:
    name = item['name']
    xmin = item['pos']['x']
    ymin = item['pos']['y']
    xmax = item['pos']['x'] + item['pos']['w']
    ymax = item['pos']['y'] + item['pos']['h']

    value = (image, width, height, name, xmin, ymin, xmax, ymax)
    csv_list.append(value)

#almacenar en CSV con el nombre de las siguientes columnas.
column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
csv_df = pd.DataFrame(csv_list, columns = column_name)

#convertir el dataframe a CSV.
csv_df.to_csv('./{}_labels.csv'.format(type_file))

De CSV a TFRecords

Instalar la librería Object Detection de TensorFlow.

En la terminal WSL:<br>
cd cv-project<br>
git clone https://github.com/tensorflow/models.git

cd models<br>
git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af

sudo apt-get install protobuf-compiler python-pil python-lxml python-tk<br>
pip install Cython contextlib2 pillow lxml matplotlib<br>
pip install pycocotools

cd research<br>
protoc object_detection/protos/*.proto --python_out=.

In [8]:
%cd ../models/research
os.environ['PYTHONPATH'] = '../models/research/:../models/research/slim/'

/mnt/c/Users/personal/Desktop/workspace/cv-project/models/research


In [10]:
#tomar el script base y modificar según el proyecto.
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import io
import tensorflow as tf
import sys
sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

def class_text_to_int(row_label):
    if row_label == 'auto': 
      return 1
    elif row_label == 'moto':
      return 2
    else:
        None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

#ejecutar dos veces, una para test y otra para train.
output_path = 'train.record' 
image_dir = '/mnt/c/Users/personal/Desktop/workspace/cv-project/p1_obtener_y_procesar_datos/dataset_original/dataset_original'
csv_input = '/mnt/c/Users/personal/Desktop/workspace/cv-project/p1_obtener_y_procesar_datos/train_labels.csv'
#solo acá no me acepta las rutas relativas, por alguna razón.

writer = tf.io.TFRecordWriter(output_path)
path = os.path.join(image_dir)
examples = pd.read_csv(csv_input)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(os.getcwd(), output_path)
print('Successfully created the TFRecords: {}'.format(output_path))

Successfully created the TFRecords: /mnt/c/Users/personal/Desktop/workspace/cv-project/models/research/train.record


Esto termina con train.record y test.record en models/research. 